# Extract data from iPyrad

The following takes `*.stats.txt` files (which are output from iPyrad) and produces three files: a `_SNPdist.csv` file, which contains numbers of variable and parsimony-informative sites, a `_sumstats.csv` file, which contains numbers of clusters, consensus reads, and loci in assembly for each sample, and finally a `_coverage.csv` file, which contains stats about coverage/sequencing depth.

For each sampling depth, it merges all .csv files into single files, either `samplingdepth_ddrad_sumstats.csv`, `samplingdepth_ddrad_snpdist.csv`, or `samplingdepth_ddrad_coverage.csv`.

Finally, it does the exact same process as above but instead of differing sampling depths, it differs clustering threshold values; end files are outputted as `clustthreshold_ddrad_sumstats.csv`, `clustthreshold_ddrad_snpdist.csv`, `clustthreshold_ddrad_coverage.csv`.

# Load relevant modules

In [ ]:
import pandas as pd
import numpy
import itertools
import csv
import glob

# Test that glob is working

The following is just a check step to make a list of file names we're going to iterate through using glob.
The `stats.txt` files are the ones we're interested in.

In [ ]:
ranafilenames = glob.glob('sampling_depth/ranaddrad*outfiles/*stats.txt')
print ranafilenames

# Define directory

Define the directory for which you want all your files produced

In [14]:
#directory='sampling_depth/*outfiles/*stats.txt' # for sampling depths t1, t2, t3, total
directory='clust_threshold/*outfiles/*stats.txt' # for clustering threshold iterations 80-95

# SNP distributions

The following pulls info about the number of variable and parsimony-informative (as iPyrad calculates it) sites and produces `_SNPdist.csv` file

In [ ]:
# Function to grab file names
def get_file_list(directory):
    return glob.glob(directory)

# Function to reference line numbers to pull out specific sections of the stats file
def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'var  sum_var' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#            print line
        elif '## Final' in line:
            endline=counter
#            print name,endline
#            print line
        else:
            continue
    infile.close()
    return [varline,endline]

# Function to cut out the SNP distribution section
def slice_data(start,end,filename):
    snp_dist=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end-3):
        lines2 = lines.strip().split()
#        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        snp_dist.append(lines2)
    infile.close()
    return snp_dist
        
# Function to convert the output to an actual dataframe format that we can read into R
def pd_conversion(filename):
    nums = get_line_numbers(filename)
    snp_dist = slice_data(nums[0],nums[1],filename)
    snpdist_labels=['number','variable','sum_var','pis', 'sum_pis', 'file_name']
    df_snpdist = pd.DataFrame.from_records(snp_dist, columns=snpdist_labels)
    return df_snpdist  

def main():
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv(filename+"_snpdist.csv")
    return dfs

print main()

# Summary statistics

The following pulls info about the number of clusters, consensus reads, and loci in assembly for each sample and produces `_sumstats.csv` file

In [ ]:
def get_file_list(directory):
    return glob.glob(directory)

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'state  reads_raw' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#        print line
        else:
            continue
    infile.close()
    return [varline,varline]

def slice_data(start,end,filename):
    sum_stats=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end+12):
        lines2 = lines.strip().split()
        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        sum_stats.append(lines2)
    infile.close()
    return sum_stats
        

def pd_conversion(filename):
    nums = get_line_numbers(filename)
    sum_stats = slice_data(nums[0],nums[1],filename)
    sumstats_labels=['sample', 'state', 'reads_raw', 'reads_passed', 'clust_total', 'clust_hidepth','hetero_est','error_est','reads_consens','loci_assembly','file_name']
    df_sumstats = pd.DataFrame.from_records(sum_stats, columns=sumstats_labels)
    return df_sumstats  

def main():
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv("./" +filename+ "_sumstats.csv")
    return dfs

print main()

# Coverage information

The following pulls info about the number of loci for which N taxa have data and produces `_coverage.csv` file

In [ ]:
def get_file_list(directory):
    return glob.glob(directory)

def get_line_numbers(filename):
    counter=0
    infile=open(filename, 'rt')
    for line in infile:
        counter+=1
        if 'locus_coverage' in line:
            varline=counter
#            print name,varline # not necessary but want to just check
#        print line
        elif '## The distribution' in line:
            endline=counter
            print filename,endline
        else:
            continue
    infile.close()
    return [varline,endline]

def slice_data(start,end,filename):
    coverage=[]
    infile=open(filename,'rU')
    for lines in itertools.islice(infile, start, end-3):
        lines2 = lines.strip().split()
        print lines2
        lines2.append(filename) # eventually change to whatever file it's on
        coverage.append(lines2)
    infile.close()
    return coverage

def pd_conversion(filename):
    nums = get_line_numbers(filename)
    coverage = slice_data(nums[0],nums[1],filename)
    cov_labels=['number','locus_coverage', 'sum_coverage','file_name']
    df_cov = pd.DataFrame.from_records(coverage, columns=cov_labels)
    return df_cov  

def main():
    file_list = get_file_list(directory)
    dfs=[]
    for filename in file_list:
        print filename
        pd_df = pd_conversion(filename)
        dfs.append(pd_df)
        pd_df.to_csv("./" +filename+ "_coverage.csv")
    return dfs

print main()

# Merge dataframes into a single file

All above code will produce separate files for each of the sampling depths (or each of the clustering thresholds). We want these all to be merged into a single file for subsequent analysis.

## Sampling depths

### Summary statistics

In [11]:
# For Epipedobates
df1 = pd.read_csv("./sampling_depth/epiddrad_t1_4_outfiles/epiddrad_t1_stats.txt_sumstats.csv")
df2 = pd.read_csv("./sampling_depth/epiddrad_t2_4_outfiles/epiddrad_t2_stats.txt_sumstats.csv")
df3 = pd.read_csv("./sampling_depth/epiddrad_t3_4_outfiles/epiddrad_t3_stats.txt_sumstats.csv")
df4 = pd.read_csv("./sampling_depth/epiddrad_total_4_outfiles/epiddrad_clust_91_stats.txt_sumstats.csv")

# For Rana
df5 = pd.read_csv("./sampling_depth/ranaddrad_t1_4_outfiles/ranaddrad_t1_stats.txt_sumstats.csv")
df6 = pd.read_csv("./sampling_depth/ranaddrad_t2_4_outfiles/ranaddrad_t2_stats.txt_sumstats.csv")
df7 = pd.read_csv("./sampling_depth/ranaddrad_t3_4_outfiles/ranaddrad_t3_stats.txt_sumstats.csv")
df8 = pd.read_csv("./sampling_depth/ranaddrad_total_4_outfiles/ranaddrad_clust_91_stats.txt_sumstats.csv")

# Concatenate everything together into single file
sumstats_concat = df1.append([df2,df3,df4,df5,df6,df7,df8])
sumstats_concat.to_csv("samplingdepth_ddrad_sumstats.csv")

### SNP distributions

In [10]:
# For Epipedobates
df1 = pd.read_csv("./sampling_depth/epiddrad_t1_4_outfiles/epiddrad_t1_stats.txt_snpdist.csv")
df2 = pd.read_csv("./sampling_depth/epiddrad_t2_4_outfiles/epiddrad_t2_stats.txt_snpdist.csv")
df3 = pd.read_csv("./sampling_depth/epiddrad_t3_4_outfiles/epiddrad_t3_stats.txt_snpdist.csv")
df4 = pd.read_csv("./sampling_depth/epiddrad_total_4_outfiles/epiddrad_clust_91_stats.txt_snpdist.csv")

# For Rana
df5 = pd.read_csv("./sampling_depth/ranaddrad_t1_4_outfiles/ranaddrad_t1_stats.txt_snpdist.csv")
df6 = pd.read_csv("./sampling_depth/ranaddrad_t2_4_outfiles/ranaddrad_t2_stats.txt_snpdist.csv")
df7 = pd.read_csv("./sampling_depth/ranaddrad_t3_4_outfiles/ranaddrad_t3_stats.txt_snpdist.csv")
df8 = pd.read_csv("./sampling_depth/ranaddrad_total_4_outfiles/ranaddrad_clust_91_stats.txt_snpdist.csv")

# Concatenate everything together into single file
sumstats_concat = df1.append([df2,df3,df4,df5,df6,df7,df8])
sumstats_concat.to_csv("samplingdepth_ddrad_snpdist.csv")

### Coverage

In [12]:
# For Epipedobates
df1 = pd.read_csv("./sampling_depth/epiddrad_t1_4_outfiles/epiddrad_t1_stats.txt_coverage.csv")
df2 = pd.read_csv("./sampling_depth/epiddrad_t2_4_outfiles/epiddrad_t2_stats.txt_coverage.csv")
df3 = pd.read_csv("./sampling_depth/epiddrad_t3_4_outfiles/epiddrad_t3_stats.txt_coverage.csv")
df4 = pd.read_csv("./sampling_depth/epiddrad_total_4_outfiles/epiddrad_clust_91_stats.txt_coverage.csv")

# For Rana
df5 = pd.read_csv("./sampling_depth/ranaddrad_t1_4_outfiles/ranaddrad_t1_stats.txt_coverage.csv")
df6 = pd.read_csv("./sampling_depth/ranaddrad_t2_4_outfiles/ranaddrad_t2_stats.txt_coverage.csv")
df7 = pd.read_csv("./sampling_depth/ranaddrad_t3_4_outfiles/ranaddrad_t3_stats.txt_coverage.csv")
df8 = pd.read_csv("./sampling_depth/ranaddrad_total_4_outfiles/ranaddrad_clust_91_stats.txt_coverage.csv")

# Concatenate everything together into single file
sumstats_concat = df1.append([df2,df3,df4,df5,df6,df7,df8])
sumstats_concat.to_csv("samplingdepth_ddrad_coverage.csv")

## Clustering thresholds

### Summary statistics

Do each of the following for all dfs produced from above and for each Epipedobates and Rana.

In [19]:
# For Epipedobates
df1 = pd.read_csv("./clust_threshold/epiddrad_clust_80_outfiles/epiddrad_clust_80_stats.txt_sumstats.csv")
# do above for remaining dfs

# For Rana
df17 = pd.read_csv("./clust_threshold/ranaddrad_clust_80_outfiles/ranaddrad_clust_80_stats.txt_sumstats.csv")
# do above for remaining dfs

sumstats_concat = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,
                             df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32])
sumstats_concat.to_csv("clustthreshold_ddrad_sumstats.csv")

### SNP distributions

In [21]:
# For Epipedobates
df1 = pd.read_csv("./clust_threshold/epiddrad_clust_80_outfiles/epiddrad_clust_80_stats.txt_snpdist.csv")
# do above for remaining dfs

# For Rana
df17 = pd.read_csv("./clust_threshold/ranaddrad_clust_80_outfiles/ranaddrad_clust_80_stats.txt_snpdist.csv")
# do above for remaining dfs

snpdist_concat = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,
                             df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32])
snpdist_concat.to_csv("clustthreshold_ddrad_snpdist.csv")

### Coverage

In [20]:
# For Epipedobates
df1 = pd.read_csv("./clust_threshold/epiddrad_clust_80_outfiles/epiddrad_clust_80_stats.txt_coverage.csv")
# do above for remaining dfs

# For Rana
df17 = pd.read_csv("./clust_threshold/ranaddrad_clust_80_outfiles/ranaddrad_clust_80_stats.txt_coverage.csv")
# do above for remaining dfs

coverage_concat = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,
                             df17,df18,df19,df20,df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32])
coverage_concat.to_csv("clustthreshold_ddrad_coverage.csv")